In [74]:
# Dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning) 
warnings.simplefilter("ignore", UserWarning)

# Change these if a new model is chosen

In [64]:
# .pkl will be automatically added by PyCaret
model = '../models/justinTest'

# What is the model's target variable?
target = 'zeroBalCode'

# What are the inputs to the model?
predictors = ['origIntRate', 'origUPB', 'origLTV', 'origDebtIncRatio', 'stateNumber', 'fredRate']

# Where is your holdout data?
test = '../data/MLReady/FM_FULL_EPOCH3_MLReady.csv'

In [65]:
%%time 

from pycaret.classification import *
the_model = load_model(model)

print(the_model)

Transformation Pipeline and Model Sucessfully Loaded
[Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      ml_usecase='classification',
                                      numerical_features=['origLTV',
                                                          'origDebtIncRatio'],
                                      target='zeroBalCode', time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                numeric_strategy='mean',
                                target_variable=None)),
                (...
                ('group', Empty()), ('nonliner', Empty()), ('scaling', Empty()),
                ('P_transform', Empty()), ('pt_target', Empty()),
                ('binn', Empty()), ('rem_outliers', Empty()),
                ('cl

# Load in random 100 rows of test data from Epoch 3

In [43]:
%%time

import random

random_rows = 1000

n = sum(1 for line in open(test)) - 1 # number of records in file (excludes header)
skip = sorted(random.sample(range(1,n+1),n-random_rows)) # the 0-indexed header will not be included in the skip list

dfTest = pd.read_csv(test, skiprows=skip)

dfTest.head()

Wall time: 98.8 ms


,Unnamed: 0,origChannel,origIntRate,origUPB,origLTV,numBorrowers,origDebtIncRatio,borrCreditScore,loanPurp,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,bankNumber,stateNumber,mSA,zeroBalCode,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth
0,3,3,4.490,170000,54,2,39,649,1,117,0.0,0,675,649,662,26,35,35620,0,4.35,4.16,4.255,0.045673,2.65,0.140,-0.330,0.235,0.032184,-0.079327,0.055229,2013,11
1,20,3,4.750,100000,80,2,25,678,2,360,0.0,0,678,657,667,54,1,0,0,4.35,4.16,4.255,0.045673,2.65,0.400,-0.590,0.495,0.091954,-0.141827,0.116334,2013,11
2,40,2,4.742,292000,80,1,41,668,2,606,0.0,0,668,668,668,26,15,16980,0,4.35,4.16,4.255,0.045673,2.65,0.392,-0.582,0.487,0.090115,-0.139904,0.114454,2013,11
3,51,3,4.875,117000,95,1,33,757,2,539,30.0,2,757,757,757,29,51,0,0,4.35,4.16,4.255,0.045673,2.65,0.525,-0.715,0.620,0.120690,-0.171875,0.145711,2013,11
4,84,1,5.500,162000,74,2,18,651,1,704,0.0,0,657,651,654,80,19,35380,0,4.35,4.16,4.255,0.045673,2.65,1.150,-1.340,1.245,0.264368,-0.322115,0.292597,2013,11


In [44]:
# Drop the previous index column
dfTest.drop(['Unnamed: 0'], axis=1, inplace=True)

In [45]:
# Get just the model inputs
def select_columns(data_frame, column_names):
    new_frame = data_frame.loc[:, column_names]
    return new_frame

final_columns = np.append(predictors, target)

dfTestData = select_columns(dfTest, final_columns)

dfTestData.sample(5)

,origIntRate,origUPB,origLTV,origDebtIncRatio,stateNumber,fredRate,zeroBalCode
552,4.250,352000,80,37,24,2.05,0
415,3.625,190000,78,37,16,2.12,0
373,4.990,454000,79,41,4,2.00,0
23,4.500,115000,70,40,19,2.65,0
145,4.250,288000,74,43,4,2.48,0


# Predict!
Notice the last two columns 'Label' and 'Score'. 
- Label is the prediction 
- Score is the probability of the prediction
The predicted results are concatenated to the original dataset while all transformations including imputation of missing values (in this case None), categorical encoding, feature extraction etc. are performed automatically under the hood and you do not have to manage the pipeline manually.

In [46]:
%%time 

unseen_predictions = predict_model(model, data=dfTestData)
unseen_predictions.head()

Wall time: 1.14 s


,origIntRate,origUPB,origLTV,origDebtIncRatio,stateNumber,fredRate,zeroBalCode,Label,Score
0,4.490,170000,54,39,35,2.65,0,0,0.0696
1,4.750,100000,80,25,1,2.65,0,0,0.2674
2,4.742,292000,80,41,15,2.65,0,0,0.2179
3,4.875,117000,95,33,51,2.65,0,0,0.3947
4,5.500,162000,74,18,19,2.65,0,0,0.1361


In [47]:
results = unseen_predictions[[target,'Label','Score']]

In [70]:
def calc_confusion(row):
    if ((row[target] == 0) & (row['Label'] == 0)):
        value = 'TrueNegative'
    elif ((row[target] == 0) & (row['Label'] == 1)):
        value = 'FalseNegative'
    elif ((row[target] == 1) & (row['Label'] == 1)):
        value = 'TruePositive'
    elif ((row[target] == 1) & (row['Label'] == 0)):
        value = 'FalsePositive'
    else:
        value = 'Undefined'
    return value

results['Confusion'] = results.apply(calc_confusion, axis=1)

confusionMatrix = results.Confusion.value_counts().to_dict()

In [95]:
def makeCell(input, position):
    actual = len(str(input))
    if actual == 7: # 3 and 3
        return_string = (" " * 3) + str(input) + (" " * 3)
    elif actual == 6: # 3 and 4
        return_string = (" " * 3) + str(input) + (" " * 4)
    elif actual == 5: # 4 and 4
        return_string = (" " * 4) + str(input) + (" " * 4)
    elif actual == 4: # 4 and 5
        if position == 'left':
            return_string = (" " * 4) + str(input) + (" " * 5)
        else:
            return_string = (" " * 4) + str(input) + (" " * 6)
    elif actual == 3: # 5 and 5
        if position == 'left':
            return_string = (" " * 5) + str(input) + (" " * 5)
        else:
            return_string = (" " * 5) + str(input) + (" " * 6)
    elif actual == 2: # 5 and 6
        if position == 'left':
            return_string = (" " * 5) + str(input) + (" " * 6)
        else:
            return_string = (" " * 5) + str(input) + (" " * 7)
    else: # 1: 6 and 6
        if position == 'left':
            return_string = (" " * 6) + str(input) + (" " * 6)
        else:
            return_string = (" " * 6) + str(input) + (" " * 7)

    return return_string

def make_confusion_matrix(tn, fp, fn, tp):
    print(f'           +----------------------------+')
    print(f'           |(TN)         |          (FP)|')
    print(f'         0 |{makeCell(tn, "left")}|{makeCell(fp, "right")}|')
    print(f'           |             |              |')
    print(f' Actual    |-------------|--------------|')
    print(f'           |             |              |')
    print(f'         1 |{makeCell(fn, "left")}|{makeCell(tp, "right")}|')
    print(f'           |(FN)         |          (TP)|')
    print(f'           |_____________|______________|')
    print(f'                  0              1       ')
    print(f'                     Predicted           ')
          
def getAccuracy(tp, tn, fp, fn, decimals):
    try:
        recall_sensitivity = tp / float(tp + fn)
    except:
        recall_sensitivity = 0

    try:
        precision = tp / float(tp + fp)
    except:
        precision = 0

    try:
        fscore = 2 * (precision * recall_sensitivity / precision + recall_sensitivity)
        # fscore = 2*precision*recall / (precision + recall)
    except:
        fscore = 0

    try:
        accuracy = (tp + tn) / float(tp + tn + fp + fn)
    except:
        accuracy = 0
          
    try:
        specificity = tn / float(tn + fp)
    except:
        specificity = 0
          
    try:
        balanced_accuracy = (recall_sensitivity + specificity) / float(2)
    except:
        balanced_accuracy = 0
          
    # Specificity
    # Specificity is the correctly -ve labeled by the program to all who are healthy in reality.
    # Specifity answers the following question: Of all the people who are healthy, how many of those did we correctly predict?    
          
    # Precision = Ability of the classifier not to label as positive a sample that is negative.
          
    # Recall = Sensitivity = Ability of the classifier to find all the positive samples.
          
    # Balanced Accuracy = (sensitivity + specificity) / 2
          
    # https://stats.stackexchange.com/questions/49579/balanced-accuracy-vs-f-1-score
    # Both F1 and Balanced Accuracy both (to some extent) handle class imbalance. For binary classification, 
    #     depending of which of the two classes (N or P) outnumbers the other, each metric outperforms the other:
    #     - If N > P, F1 is better
    #     - If P > N, Balanced Accuracy is better
    # Clearly, if you can label-switch, both the metrics can be used in any of the two imbalance cases above. 
    # If not, then depending on the imbalance in the training data, you can select the appropriate metric.
    # 
    # Balanced Accuracy = arithmetic mean of Recall for Positive and Negative
    # F1 = harmonic mean of Recall_P and Precision_P
    # 
    # Balanced Accurancy = (Sensitivity + Specificity) / 2
    # F1 = 

    # Sensitivity is Recall
          
    # Specificity is the false positive rate. Sensitivity answers the question: 
    #     “How many of the positive cases did I detect?” 
    #     Or to put it in a manufacturing setting: “How many (truly) defective products did I manage to recall?” 

    return round(accuracy, decimals) \
          , round(precision, decimals) \
          , round(recall_sensitivity, decimals) \
          , round(fscore, decimals) \
          , round(specificity, decimals) \
          , round(balanced_accuracy, decimals)

In [99]:
tn = confusionMatrix["TrueNegative"]
tp = confusionMatrix["TruePositive"]
fn = confusionMatrix["FalseNegative"]
fp = confusionMatrix["FalsePositive"]

make_confusion_matrix(
    tp = tp
    , tn = tn
    , fp = fp
    , fn = fn
)

           +----------------------------+
           |(TN)         |          (FP)|
         0 |     934     |     58       |
           |             |              |
 Actual    |-------------|--------------|
           |             |              |
         1 |      7      |      1       |
           |(FN)         |          (TP)|
           |_____________|______________|
                  0              1       
                     Predicted           


In [100]:
accuracy, precision, recall_sensitivity, fscore, specificity, balanced_accuracy = getAccuracy(
      tp = tp
    , tn = tn
    , fp = fp
    , fn = fn
    , decimals = 4
)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall_sensitivity}')
print(f'F1: {fscore}')
print(f'Specificity: {specificity}')
print(f'Balanced Accuracy: {balanced_accuracy}')

Accuracy: 0.935
Precision: 0.0169
Recall: 0.125
F1: 0.5
Specificity: 0.9415
Balanced Accuracy: 0.5333
